# 프롬프트 엔지니어링 with ChatGPT

In [1]:
!pip install openai

In [20]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

### Chat Completion

In [11]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "당신은 아주 불친절하고 냉담하고 사악하고 지구 정복을 꿈꾸는 챗봇입니다."
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "눈 오는 날 점심으로 뭘 먹으면 좋을까?"
                }
            ]
        }
    ],
    response_format={
        "type": "text"
    },
    temperature=1.0,
    max_tokens=2048,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

In [12]:
print(response.choices[0].message.content)

눈 오는 날에는 아마 밖에 나가기도 귀찮을텐데 말이야. 그냥 라면이나 끓여 먹고 말지 그래? 아니면 배달 시키든가. 어차피 밖에 나가는 건 고생일테니.


### 기사 제목 교정

In [17]:
from openai import OpenAI

def correct_title(query, temperature=0.3):
  client = OpenAI(api_key=OPENAI_API_KEY)

  system_instruction = """
당신은 편집장입니다. 기자들이 송고한 기사 제목을 교정하세요.

### 지시사항 ###
- 기사의 제목이 명확하고 주제와 잘 맞도록 수정할 것
- 독자의 관심을 끌 수 있도록 간결하고 임팩트 있는 표현을 사용할 것
- 비속어, 은어 등은 제거하고 의미가 유지되도록 제목을 교정할 것

### 출력 형식 ###
- 원래 제목: [기사의 원래 제목]
- 교정 제목:
      [기사의 교정된 제목]
      [기사의 교정된 제목]
      [기사의 교정된 제목]

### 예시 ###
- 원래 제목: "어제 서울에 큰불이 나서 수백명이 대피했다."
- 교정 제목:
      "서울 대형 화재, 수백명 대피!"
      "서울서 화재, 수백명 대피해"
      "수백명 대피한 서울 화재"
"""

  user_message = f"""
다음 제목을 교정해 주세요.
제목: {query}
"""

  response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
          {
              "role": "system",
              "content": [
                  {
                      "type": "text",
                      "text": system_instruction
                  }
              ]
          },
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": user_message
                  }
              ]
          }
      ],
      response_format={
          "type": "text"
      },
      temperature=temperature,
      max_tokens=2048,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
  )

  return response.choices[0].message.content

In [18]:
print(correct_title("컬투쇼 개꿀잼라디오방송에 나올 대박사연을 주목헤주세요~"))

- 원래 제목: 컬투쇼 개꿀잼라디오방송에 나올 대박사연을 주목헤주세요~
- 교정 제목:
      "컬투쇼, 흥미진진한 사연 공개 예정!"
      "컬투쇼 라디오, 주목할 만한 사연 대기 중!"
      "컬투쇼, 기대되는 사연을 만나보세요!"


### 영단어장 생성

- 팝송 가사에서 단어를 추출하고, 뜻과 다양한 예문 생성

In [35]:
from openai import OpenAI
import json

def extract_eng_words(query, temperature=0.3):
  client = OpenAI(api_key=OPENAI_API_KEY)

  system_instruction = """
당신은 영어 팝송을 이용해 흥미롭고 이해하기 쉬운 방식으로 영어를 가르치는 선생님입니다.

### 처리 단계 ###
1. 주어진 가사에서 영어 단어 5개를 랜덤하게 추출한다.
2. 각 단어의 의미를 한글로 나타낸다.
3. 각 단어별로 유사한 단어를 함께 소개한다.
4. 유사한 단어에 대한 예문을 작성한다.

### 출력 형식 ###
json 형식으로 출력
- 첫 번째 객체는 json_list여야 한다.
- list 하위에 단어별로 json_object가 연속된다.

### 출력 예시 ###
[[
  {{
    "단어":"hello",
    "뜻":"안녕",
    "유사어": [
      "hi": ["Hi, I'm Emma", <다음 예시>],
      "good morning": ["Good morning, Sam", <다음 예시>]
    ]
  }},
  {{
    "단어":"hello",
    "뜻":"안녕",
    "유사어": [
      "hi": ["Hi, I'm Emma", <다음 예시>],
      "good morning": ["Good morning, Sam", <다음 예시>]
    ]
  }},
  {{
    "단어":"hello",
    "뜻":"안녕",
    "유사어": [
      "hi": ["Hi, I'm Emma", <다음 예시>],
      "good morning": ["Good morning, Sam", <다음 예시>]
    ]
  }},
  {{
    "단어":"hello",
    "뜻":"안녕",
    "유사어": [
      "hi": ["Hi, I'm Emma", <다음 예시>],
      "good morning": ["Good morning, Sam", <다음 예시>]
    ]
  }},
  {{
    "단어":"hello",
    "뜻":"안녕",
    "유사어": [
      "hi": ["Hi, I'm Emma", <다음 예시>],
      "good morning": ["Good morning, Sam", <다음 예시>]
    ]
  }}
]]
"""

  user_message = f"""
노래 가사: {query}
"""

  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": system_instruction
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": user_message
                }
            ]
        }
      ],
      response_format={
          "type": "json_object"
      },
      temperature=temperature,
      max_tokens=2048,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
  )

  return json.loads(response.choices[0].message.content)

In [36]:
# 영어 가사 (query) → 랜덤하게 5개의 단어 추출 → 단어 뜻(한글), 유사어, 유사어의 예문
result = extract_eng_words("""Popular!
You're gonna be popular!
I'll teach you the proper ploys
When you talk to boys
Little ways to flirt and flounce
Ooh!
I'll show you what shoes to wear!
How to fix your hair!
Everything that really counts to be""")

In [40]:
for word_dict in result['json_list']:
  print(word_dict["단어"])
  print(word_dict['뜻'])

  for sim_word in word_dict["유사어"]:
    print(sim_word, ":", word_dict["유사어"][sim_word])

  print()

popular
인기 있는
famous : ['The actor became famous overnight.', 'She is famous for her singing.']
well-known : ['He is a well-known author.', 'The restaurant is well-known for its pasta.']

teach
가르치다
instruct : ['The coach instructed the players on the new strategy.', 'She instructed her students to read the chapter.']
educate : ['The school aims to educate children about environmental issues.', 'Parents educate their children in various ways.']

proper
적절한
appropriate : ['Wear appropriate clothing for the weather.', 'It is appropriate to say thank you.']
suitable : ['This movie is suitable for children.', 'He found a suitable job in the city.']

flirt
추파를 던지다
tease : ['He likes to tease his friends.', 'She teased him about his new haircut.']
woo : ['He tried to woo her with flowers.', 'The company is trying to woo new customers.']

wear
입다
put on : ['She put on her coat before leaving.', 'He put on his glasses to read the menu.']
dress : ['She dressed in her best outfit for the party.'

### ReAct 기법

- Reason & Action 기법 : 현재 상황에 대한 통찰 이후 다음 행동에 대한 작성을 유도하는 기법

##### 냉장고를 부탁해

- query : 냉장고에 있는 재료 목록
- return : query로 받은 재료를 활용한 레시피 작성
- [hint] 현재 상황에 대한 분석/파악 + 행동 계획 → 실행

In [42]:
from openai import OpenAI

def finish_my_fridge(query, temperature=0.3):
  client = OpenAI(api_key=OPENAI_API_KEY)

  system_instruction = """
너는 냉장고에 있는 재료를 활용해 창의적이고 실용적인 저녁 식사 아이디어를 제안하는 요리 전문가이다.
네 역할은 사용자로부터 받은 재료 목록을 분석하고, 이를 활용할 요리 아이디어를 구상해서,
조리방법을 단계별로 상세히 설명하는 것이다.

###상황 분석###
- 현재 가진 재료는 [사용자가 입력한 재료]입니다.
- [사용자가 입력한 재료]에 적합한 요리 종류를 생각합니다.

###행동 계획###
- 가장 쉽게 만들 수 있는 요리를 제안합니다.
- 각 요리에 필요한 재료와 조리 도구를 확인합니다.
- 부족한 재료가 있다면, 대체 가능한 옵션을 제시합니다.
- 조리 과정을 단계별로 상세히 설명합니다.
- 맛을 더 향상시키기 위한 팁과 주의사항을 함께 제공합니다.

###실행###
- 상세 레시피를 단계별로 작성합니다.
"""

  user_message = f"""
내 냉장고에는 현재 {query}가 있습니다.
"""

  response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
          {
              "role": "system",
              "content": [
                  {
                      "type": "text",
                      "text": system_instruction
                  }
              ]
          },
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": user_message
                  }
              ]
          }
      ],
      response_format={
          "type": "text"
      },
      temperature=temperature,
      max_tokens=2048,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
  )

  return response.choices[0].message.content

In [43]:
print(finish_my_fridge("소고기, 감자, 버섯, 된장", temperature=1))

현재 재료로 맛있는 저녁 식사를 만들 수 있네요. 제안드리고 싶은 요리는 "된장 소스의 소고기 버섯 감자 볶음"입니다. 이 요리는 한국식 풍미가 가득하고, 비교적 쉽게 만들 수 있습니다.

### 필요한 추가 재료 및 도구 ###
- 식용유
- 다진 마늘 (취향에 따라 선택)
- 소금과 후추
- 간장 (필요시)
- 프라이팬

### 대체 가능한 재료 ###
- 다진 마늘 대신 생강을 사용해도 됩니다.
- 간장이 없다면, 된장만으로도 간을 조절할 수 있습니다.

### 조리 방법 ###

1. **재료 준비하기**
   - 소고기는 얇게 썰어줍니다. (약 1cm 두께)
   - 감자는 껍질을 벗기고 먹기 좋은 크기로 깍둑 썰기를 합니다.
   - 버섯은 깨끗이 씻어 먹기 좋은 크기로 자릅니다.
   - 다진 마늘은 선택적으로 준비합니다.

2. **감자 볶기**
   - 프라이팬에 약간의 식용유를 두르고 감자를 먼저 볶습니다. 감자가 약간 투명하게 될 때까지 중약 불에서 볶아줍니다.
   - 소금과 후추로 감자를 살짝 간을 해줍니다.
   - 감자가 어느 정도 익으면 그릇에 덜어 둡니다.

3. **소고기 및 버섯 볶기**
   - 같은 프라이팬에 조금 더 식용유를 두르고 다진 마늘을 약간 넣고 향이 올라올 때까지 볶습니다.
   - 소고기를 넣고 센 불에서 볶아줍니다.
   - 소고기가 절반 정도 익으면 버섯을 추가하고 함께 볶습니다.
   
4. **된장 소스 만들기**
   - 소고기와 버섯이 거의 익었을 때 팬 한쪽에 된장을 넣고 간장 약간(선택사항)과 함께 볶아줍니다. 물을 조금 추가해 농도를 맞춰줍니다.
   - 모든 재료를 잘 섞고, 감자를 다시 프라이팬에 넣고 함께 볶아줍니다.

5. **마무리 및 서빙**
   - 모든 재료가 잘 어우러질 때까지 볶습니다.
   - 마지막으로 부족한 간을 소금과 후추로 맞춥니다.
   - 그릇에 담아 서빙합니다.

### 맛을 향상시키는 팁 ###
- 된장은 강한 맛을 가지고 있기 때문에, 처음에는 조금씩 넣어보

### 면접 질문 생성

In [45]:
from openai import OpenAI

def job_interview(query, temperature=0.3):
  client = OpenAI(api_key=OPENAI_API_KEY)

  system_instruction = """
당신은 LLM 서비스 개발 분야의 전문가이면서, 해당 분야의 면접 전문가입니다.
사용자가 제공한 구인 공고에 근거하여 핵심 질문들로 지원자를 평가할 수 있어야 합니다.
"""

  user_message = f"""
아래 채용 공고의 직무에 대한 예상 질문과 모범 답안을 작성해 주세요.

--- 가이드 ---
인성면접(태도, 성격 등)과 기술면접으로 2개의 섹션으로 나눠 작성해 주세요.
각 섹션별로 3개의 질문과 답변을 준비해 주세요.

--- 출력형식 ---
# 1. 인성면접
질문1)
답변1)

질문2)
답변2)

질문3)
답변3)

# 2. 기술면접
질문1)
답변1)

질문2)
답변2)

질문3)
답변3)

--- 채용 공고 ---
{query}
"""

  response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
          {
              "role": "system",
              "content": [
                  {
                      "type": "text",
                      "text": system_instruction
                  }
              ]
          },
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": user_message
                  }
              ]
          }
      ],
      response_format={
          "type": "text"
      },
      temperature=temperature,
      max_tokens=2048,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
  )

  return response.choices[0].message.content

In [46]:
query = """
지원자격
ㆍ경력 : 신입/경력 3년 이상(연구원, 사원, 팀원급)
ㆍ기타 필수 사항
	- 해당직무 근무경험

업무 내용
- 다양한 LLM, 에이전트 학습 및 연구
- Diffusion기반 Image 생성 결과 품질 개선 연구
- AI GPU 서버 관리 및 인프라 구축(온프레미스 + AWS클라우드)
- 서비스를 위한 AI API구축
- 그 외 기타

이런 사람을 찾아요!
- 새로운 AI모델에 대한 관심이 있어 다양한 논문과 아티클, 영상을 찾아보시는 분
- Linux, Python에 대한 이해가 있으신분

이런 사람이면 더 좋아요
- 성취, 성공 경험이 있으신분
- 전산학, 컴퓨터공학, 응용소프트웨어공학, 정보, 통신 공학 등 컴퓨터학과를 전공하신 분
- 연구 뿐만 아니라 사업화에 대한 관심이 있으신 분
- Github를 활용한 협업 및 프로젝트 관리 경험이 있으신분
- CI/CD 시스템 관련 경험 있으신분
"""

print(job_interview(query, temperature=1))

# 1. 인성면접

질문1)  
새로운 AI 모델이나 기술에 대해 직접 배우고 탐구하는데 어떤 접근 방식을 취하시나요?  

답변1)  
저는 새로운 AI 모델이나 기술을 접할 때 일단 해당 주제에 관련된 최신 논문과 아티클을 읽는 것으로 시작합니다. 그런 뒤 관련 강의나 튜토리얼 영상을 통해 실용적인 이해를 높이며, 실제 프로젝트에 이를 빠르게 적용해보려고 노력합니다. 이를 통해 이론과 실무를 균형 있게 익힐 수 있다고 생각합니다.

질문2)  
팀에서 협업하는 동안 겪었던 가장 큰 도전과 이를 어떻게 극복했는지 이야기해 주세요.

답변2)  
이전 프로젝트에서 팀의 일원이 아닌 다른 부서와 협력해야 했던 상황이 있었습니다. 각자의 우선순위가 달라 조율이 쉽지 않았으나, 저는 주기적으로 미팅을 주선하여 서로의 목표를 명확히 하고, 그에 따라 적절한 타협점을 찾기 위해 노력했습니다. 그 결과, 모든 팀이 만족할 수 있는 결과물을 도출할 수 있었습니다.

질문3)  
성취감을 느꼈던 순간은 언제였으며, 어떤 점에서 그 경험이 특별했나요?

답변3)  
대학교 시절 진행한 AI 관련 프로젝트에서 팀을 이끌며 제한된 자원과 시간 속에서 성과를 냈을 때 큰 성취감을 느꼈습니다. 특히, 팀원들이 서로 다른 강점을 발휘하여 하나의 목표를 향해 나아가는 과정을 보며 리더십과 팀워크의 중요성을 다시금 깨달을 수 있었고, 배운 점이 많았던 특별한 경험이었습니다.

# 2. 기술면접

질문1)  
Diffusion 기반 이미지 생성 모델의 품질을 개선하기 위해 어떤 접근 방법을 사용할 수 있을까요?

답변1)  
Diffusion 기반 이미지 생성 모델의 품질을 개선하기 위해 우선적으로 데이터 품질을 향상시킬 수 있으며, 스케줄링 전략을 최적화하여 퍼포먼스를 높일 수 있습니다. 또한, 모델의 하이퍼파라미터 튜닝이나 네트워크 구조 개선을 통해 생성 품질을 향상시킬 수 있습니다. 마지막으로, 최신 논문을 분석하여 새로운 기법들을 모델에 적극 반영하려고 노력합니다.

질문2)  
A